In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import time
import plotly
from datetime import datetime
import optuna
import sklearn
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
SEED = 10
limit_rows = None

In [ ]:
print(f"Execution started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

In [ ]:
def load_csv(name):
    if limit_rows is None:
        full_filename = f'../Data Preprocessing/sklearn/full/iot23_combined_{name}.csv'
    else:
        full_filename = f'../Data Preprocessing/sklearn/semi/iot23_combined_{int(limit_rows/1000)}k_{name}.csv'
    
    df = pd.read_table(filepath_or_buffer=full_filename, header=None, sep=',').infer_objects().to_numpy()
    
    return df.ravel() if df.shape[1] == 1 else df

In [ ]:
X_train, X_test, y_train, y_test = load_csv('X_train'), load_csv('X_test'), load_csv('y_train'), load_csv('y_test')

print('X_train',X_train.shape,'\ny_train',y_train.shape)
print('X_test',X_test.shape,'\ny_test',y_test.shape)

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

n_features = X_train.shape[1]
n_classes = len(set(y_train))

def objective(trial):
    
    reg_param = trial.suggest_discrete_uniform("reg_param", 0.1, 1.0, 0.1)

    classifier_obj = QuadraticDiscriminantAnalysis(reg_param=reg_param)
    
    classifier_obj.fit(X_train, y_train)
    
    y_pred = classifier_obj.predict(X_test)
    
    accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
    
    return accuracy

In [ ]:
print(f"Optimization started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

In [ ]:
study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=100, n_jobs=-1, catch=(UserWarning,))

In [ ]:
print(f"Optimization finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

In [ ]:
def format_trial(trial):
    return json.dumps({'number' : trial.number,
                       'score'  : trial.values[0],
                       'params' : trial.params}, indent=4, default=str)

In [ ]:
print(format_trial(study.best_trial))

In [ ]:
for trial in study.best_trials:
    print(format_trial(trial))

In [ ]:
optuna.visualization.plot_optimization_history(study).show()